Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import time
import os
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

# TORCH
import torch
import torchvision
from torch.autograd import Variable
from torch.utils.data import TensorDataset,DataLoader
from torchvision import models, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1234)

Load Data

In [ ]:
Xi = pd.read_csv("expression1.csv")
yi = pd.read_csv("class.csv")
Xi = np.array(Xi)
X = np.transpose(Xi)
yi = np.array(yi)
yi = np.concatenate(yi)

In [ ]:
#Preprocessing dataset using Min Max technique
X_min = X.min()
X_max = X.max()
X_range = (X_max- X_min)
X_scaled = (X - X_min)/(X_range)
X = X_scaled

In [ ]:
#create a 2d matrix for furthur processing
def split_sequence(x,n_steps_in):
    end_ix = 0
    final = []
    while(len(x)>end_ix):
        X = []
        for item in x[end_ix:n_steps_in]:
            X.append(item)
        end_ix = end_ix + 50
        n_steps_in = n_steps_in + 50 
        final.append(X)
    w=np.asarray(final)
    return w
x = []
n_steps_in = 50

for k in X:
    X_0 = split_sequence(k, n_steps_in)
    x.append(X_0)

In [ ]:
#divide dataset into 57 samples for training and 5 samples for testing
X_train, X_test, y_train, y_test = train_test_split(X, yi, test_size = 0.08)
ltrain=len(X_train)
ltest=len(X_test)
#Reshape the samples for tensor
X_train.shape=(ltrain,1,40,50)
y_train.shape=(ltrain)
X_test.shape=(ltest,1,40,50) 
y_test.shape=(ltest)
# Convert train data into tensors and Wrap them in Variable
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).long()
# Convert test data into tensors and wrap them in Variable
X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).long()

In [ ]:
print(X_train.size())
print(X_test.size())
print(y_train.size())
print(y_test.size())

In [ ]:
# Check availability of GPU
use_gpu = torch.cuda.is_available()
if use_gpu:
    print('GPU is available!')

Define Network architecture

In [ ]:
class double_conv(nn.Module):
    '''(conv => ReLU) * 2'''
    def __init__(self, in_ch, out_ch):
        super(double_conv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.ReLU(inplace=True)
        )
        
    def forward(self, x):
        x = self.conv(x)
        return x
    
class inconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(inconv, self).__init__()
        self.conv = double_conv(in_ch, out_ch)
    
    def forward(self, x):
        x = self.conv(x)
        return x

class down(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(down, self).__init__()
        self.mpconv = nn.Sequential(
            nn.MaxPool2d(2),
            double_conv(in_ch, out_ch)
        )

    def forward(self, x):
        x = self.mpconv(x)
        return x

class up(nn.Module):
    def __init__(self, in_ch, out_ch, bilinear=True):
        super(up, self).__init__()
        if bilinear:
            self.up = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)
        else:
            self.up = nn.ConvTranspose2d(in_ch//2, in_ch//2, 2, stride=2)
        self.conv = double_conv(in_ch, out_ch)

    def forward(self, x1, x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]
        x1 = F.pad(x1, (diffX // 2, diffX - diffX//2,
                        diffY // 2, diffY - diffY//2))
        x = torch.cat([x2, x1], dim=1)
        x = self.conv(x)
        return x

class outconv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super(outconv, self).__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 1)

    def forward(self, x):
        x = self.conv(x)
        return x

In [ ]:
#assembly of the sub-parts to form the complete net
class UNet(nn.Module):
    def __init__(self, n_channels, n_classes):
        super(UNet, self).__init__()
        self.inc = inconv(n_channels, 64)
        self.down1 = down(64, 128)
        self.down2 = down(128, 256)
        self.down3 = down(256, 512)
        self.down4 = down(512, 512)
        self.up1 = up(1024, 256)
        self.up2 = up(512, 128)
        self.up3 = up(256, 64)
        self.up4 = up(128, 64)
        self.outc = outconv(64, n_classes)
        
    def forward(self, x):
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        xf = x
        x = self.outc(x)
        x = x.view(-1,self.num_flat_features(x))
        return x,xf
    
    def num_flat_features(self,x):
        size=x.size()[1:]
        num_features=1
        for s in size:
            num_features *= s
        return num_features

Initializing the network

In [ ]:
net = UNet(n_channels=1, n_classes=2)
if use_gpu:
    net = net.cuda()

Define Loss Function and Optimizer

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=1e-3)

Train the network

In [ ]:
iterations = 100
trainLoss = []
testLoss = []
start = time.time()

for epoch in range(iterations):
    epochStart = time.time()
    runningLoss = 0   
    net.train(True) # For training
    # Initialize gradients to zero
    optimizer.zero_grad()
    # Feed-forward input data through the network
    out,ft = net(X_train)
    out = F.log_softmax(out,dim=1)
    loss = criterion(out, y_train)
    # Backpropagate loss and compute gradients
    loss.backward()
    # Update the network parameters
    optimizer.step()
    # Accumulate loss
    runningLoss += loss.item()
    avgTrainLoss = runningLoss/ltrain    
    trainLoss.append(avgTrainLoss)
    
    # Evaluating performance on test set for each epoch
    net.train(False) # For testing
    test_runningLoss = 0    
    outputs,q = net(X_test)       
    # Compute loss/error
    loss = criterion(F.log_softmax(outputs,dim=1), y_test)      
    # Accumulate loss per batch
    test_runningLoss += loss.item() 
    avgTestLoss = test_runningLoss/ltest    
    testLoss.append(avgTestLoss)
        
    # Plotting Loss vs Epochs
    fig1 = plt.figure(1)        
    plt.plot(range(epoch+1),trainLoss,'r--',label='train')        
    plt.plot(range(epoch+1),testLoss,'g--',label='test')        
    if epoch==0:
        plt.legend(loc='upper left')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')   
   
    epochEnd = time.time()-epochStart
    print('At Iteration: {:.0f} /{:.0f}  ;  Training Loss: {:.6f}; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTrainLoss,epochEnd//60,epochEnd%60))
    
    print('At Iteration: {:.0f} /{:.0f}  ;  Testing Loss: {:.6f} ; Time consumed: {:.0f}m {:.0f}s '\
          .format(epoch + 1,iterations,avgTestLoss,epochEnd//60,epochEnd%60))
    
end = time.time()-start
print('Training completed in {:.0f}m {:.0f}s'.format(end//60,end%60))


In [ ]:
#get prediction 
segImg,g = net(Variable(X_test))
# Applying softmax to get class probabilities
segImg_np = F.log_softmax(segImg,dim=1).data.cpu().squeeze(1)
_, predicted = torch.max(segImg_np, 1) 
print(predicted)
print(y_test)
print('Accuracy of the network %d %%' % (100 * torch.sum(y_test==predicted) / ltest))

In [ ]:
predicted=predicted.data.numpy()
cm = np.array(confusion_matrix(ytest, predicted, labels=[1,0]))
confusion = pd.DataFrame(cm, index=['is_cancer', 'is_healthy'],
                         columns=['predicted_cancer','predicted_healthy'])
confusion

In [ ]:
sns.heatmap(confusion,annot=True,fmt="d")
print(classification_report(y_test,predicted))

In [ ]:
f00 = ft[0][0].detach().numpy()
np.savetxt("f00.csv",f00)
f01 = ft[0][1].detach().numpy()
np.savetxt("f01.csv",f01)
f02 = ft[0][2].detach().numpy()
np.savetxt("f02.csv",f02)
f03 = ft[0][3].detach().numpy()
np.savetxt("f03.csv",f03)
f04 = ft[0][4].detach().numpy()
np.savetxt("f04.csv",f04)
f05 = ft[0][5].detach().numpy()
np.savetxt("f05.csv",f05)
f06 = ft[0][6].detach().numpy()
np.savetxt("f06.csv",f06)
f07 = ft[0][7].detach().numpy()
np.savetxt("f07.csv",f07)
f08 = ft[0][8].detach().numpy()
np.savetxt("f08.csv",f08)
f09 = ft[0][9].detach().numpy()
np.savetxt("f09.csv",f09)
f010 = ft[0][10].detach().numpy()
np.savetxt("f010.csv",f010)
f011 = ft[0][11].detach().numpy()
np.savetxt("f011.csv",f011)
f012 = ft[0][12].detach().numpy()
np.savetxt("f012.csv",f012)
f013 = ft[0][13].detach().numpy()
np.savetxt("f013.csv",f013)
f014 = ft[0][14].detach().numpy()
np.savetxt("f014.csv",f014)
f015 = ft[0][15].detach().numpy()
np.savetxt("f015.csv",f01)
f016 = ft[0][16].detach().numpy()
np.savetxt("f016.csv",f016)
f017 = ft[0][17].detach().numpy()
np.savetxt("f017.csv",f017)
f018 = ft[0][18].detach().numpy()
np.savetxt("f018.csv",f018)
f019 = ft[0][19].detach().numpy()
np.savetxt("f019.csv",f019)
f020 = ft[0][20].detach().numpy()
np.savetxt("f020.csv",f020)
f021 = ft[0][21].detach().numpy()
np.savetxt("f021.csv",f021)
f022 = ft[0][22].detach().numpy()
np.savetxt("f022.csv",f022)
f023 = ft[0][23].detach().numpy()
np.savetxt("f023.csv",f023)
f024 = ft[0][24].detach().numpy()
np.savetxt("f024.csv",f024)
f025 = ft[0][25].detach().numpy()
np.savetxt("f025.csv",f025)
f026 = ft[0][26].detach().numpy()
np.savetxt("f026.csv",f026)
f027 = ft[0][27].detach().numpy()
np.savetxt("f027.csv",f027)
f028 = ft[0][28].detach().numpy()
np.savetxt("f028.csv",f028)
f029 = ft[0][29].detach().numpy()
np.savetxt("f029.csv",f029)
f030 = ft[0][30].detach().numpy()
np.savetxt("f030.csv",f030)
f031 = ft[0][31].detach().numpy()
np.savetxt("f031.csv",f031)
f032 = ft[0][32].detach().numpy()
np.savetxt("f032.csv",f032)
f033 = ft[0][33].detach().numpy()
np.savetxt("f033.csv",f033)
f034 = ft[0][34].detach().numpy()
np.savetxt("f034.csv",f034)
f035 = ft[0][35].detach().numpy()
np.savetxt("f035.csv",f035)
f036 = ft[0][36].detach().numpy()
np.savetxt("f036.csv",f036)
f037 = ft[0][37].detach().numpy()
np.savetxt("f037.csv",f037)
f038 = ft[0][38].detach().numpy()
np.savetxt("f038.csv",f038)
f039 = ft[0][39].detach().numpy()
np.savetxt("f039.csv",f039)
f040 = ft[0][40].detach().numpy()
np.savetxt("f040.csv",f040)
f041 = ft[0][41].detach().numpy()
np.savetxt("f041.csv",f041)
f042 = ft[0][42].detach().numpy()
np.savetxt("f042.csv",f042)
f043 = ft[0][43].detach().numpy()
np.savetxt("f043.csv",f043)
f044 = ft[0][44].detach().numpy()
np.savetxt("f044.csv",f044)
f045 = ft[0][45].detach().numpy()
np.savetxt("f045.csv",f045)
f046 = ft[0][46].detach().numpy()
np.savetxt("f046.csv",f046)
f047 = ft[0][47].detach().numpy()
np.savetxt("f047.csv",f047)
f048 = ft[0][48].detach().numpy()
np.savetxt("f048.csv",f048)
f049 = ft[0][49].detach().numpy()
np.savetxt("f049.csv",f049)
f050 = ft[0][50].detach().numpy()
np.savetxt("f050.csv",f050)
f051 = ft[0][51].detach().numpy()
np.savetxt("f051.csv",f051)
f052 = ft[0][52].detach().numpy()
np.savetxt("f052.csv",f052)
f053 = ft[0][53].detach().numpy()
np.savetxt("f053.csv",f053)
f054 = ft[0][54].detach().numpy()
np.savetxt("f054.csv",f054)
f055 = ft[0][55].detach().numpy()
np.savetxt("f055.csv",f055)
f056 = ft[0][56].detach().numpy()
np.savetxt("f056.csv",f056)
f057 = ft[0][57].detach().numpy()
np.savetxt("f057.csv",f057)
f058 = ft[0][58].detach().numpy()
np.savetxt("f058.csv",f058)
f059 = ft[0][59].detach().numpy()
np.savetxt("f059.csv",f059)
f060 = ft[0][60].detach().numpy()
np.savetxt("f060.csv",f060)
f061 = ft[0][61].detach().numpy()
np.savetxt("f061.csv",f061)
f062 = ft[0][62].detach().numpy()
np.savetxt("f062.csv",f062)
f063 = ft[0][63].detach().numpy()
np.savetxt("f063.csv",f063)